Zagadnieniem tej pracy domowej było wybranie bazy danych oraz zbudowanie modeli klasyfikacji do niej, tak aby sprawdzić ich działanie. Moja baza danych opisywała wiele różnych
parametrów wina, takich jak kwasowość, cukry, siarczyny oraz alkohol i tym podobne oraz jego ocena. Zadaniem było stworzenie modelu, który na podstawie tych parametrów będzie
w stanie określić, jaką ocena wino otrzymało. W tym celu wykorzystałem 4 różne modele: drzewo decyzyjne, k-Najbliższych sąsiadów, Naive Bayes oraz sieć neuronową. Wszystkie
modele zostały przetestowane na danych treningowych, a następnie na danych testowych. Przetestowane zostało wiele parametrów danych modeli.

Funkcje odpowiadające za wczytanie danych i ich preprocessing:

In [11]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd
from sklearn.utils import shuffle

def getData(directory):
    data = pd.read_csv(directory)
    data = shuffle(data)
    data['quality'] = data['quality'].replace([3, 4, 5], 'low')
    data['quality'] = data['quality'].replace([6, 7], 'medium')
    data['quality'] = data['quality'].replace([8, 9], 'high')
    return data

def withDeletion(df):
    label_encoder = LabelEncoder()
    df["type"] = label_encoder.fit_transform(df["type"])
    df = df.dropna()
    return df

def withMean(df):
    label_encoder = LabelEncoder()
    df["type"] = label_encoder.fit_transform(df["type"])
    df = df.fillna(df.mean(numeric_only=True))
    return df

def normalization(df):
    scaler = MinMaxScaler()
    columns_to_normalize = df.columns[:-1]
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
    return df

print("Wczytane i pomieszane dane: ")
print(getData("./Data/winequality.csv").head())
print("Dane po normalizacji: ")
print(normalization(withMean(getData("./Data/winequality.csv"))).head())

Wczytane i pomieszane dane: 
       type  fixed acidity  volatile acidity  citric acid  residual sugar  \
3475  white            6.2              0.36         0.45            10.4   
1137  white            6.1              0.31         0.37             8.4   
4460  white            6.6              0.18         0.26            17.3   
4508  white            5.8              0.26         0.30             2.6   
4247  white            6.6              0.36         0.47             1.4   

      chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
3475      0.060                 22.0                 184.0  0.99711  3.31   
1137      0.031                 70.0                 170.0  0.99340  3.42   
4460      0.051                 17.0                 149.0  0.99840  3.00   
4508      0.034                 75.0                 129.0  0.99020  3.20   
4247      0.145                 26.0                 124.0  0.99274  3.09   

      sulphates  alcohol quality  
3475      

Są tu dwie funkcje odpowiadające za preprocessing danych. Pierwsza z nich usuwa wszystkie wiersze, które zawierają jakąś wartość NaN. Druga z nich zastępuje wartości NaN średnią.

Pierwszym z modeli jest Drzewo Decyzyjne, ma ono 2 wersje, 1 gdzie maksymalny poziom wynosi 9, a druga gdzie maksymalny poziom drzewa wynosi 5. Kod wygląda następująco:

In [12]:
from sklearn.tree import DecisionTreeClassifier

def classify(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size)
    classifier = DecisionTreeClassifier(criterion="entropy", max_depth=9, min_samples_leaf=2, splitter="best",
                                        class_weight=None, random_state=101)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix


def classifyMax(df, train_size, max_depth):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)
    classifier = DecisionTreeClassifier(max_depth=max_depth)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [13]:
from Methods import DecisionTree
from Methods import Preprocessing

TRAIN_SIZE = 0.9


def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfDT(dataD, dataM)



def printResult(result):
    print("Accurarcy: ", result[0])
    # print("Confusion Matrix: ")
    # print(result[1])
    return ""


def getResultsOfDT(dataD, dataM):
    print("Without max-depth: ")
    print("Without normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classify(dataD, TRAIN_SIZE)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classify(dataM, TRAIN_SIZE)), end='')
    print("With normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classify(Preprocessing.normalization(dataD), TRAIN_SIZE)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classify(Preprocessing.normalization(dataM), TRAIN_SIZE)), end='')
    print("With max-depth: ")
    print("Without normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classifyMax(dataD, TRAIN_SIZE, 5)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classifyMax(dataM, TRAIN_SIZE, 5)), end='')
    print("With normalization:")
    print("DT w/ Deletion: ")
    print(printResult(DecisionTree.classifyMax(Preprocessing.normalization(dataD), TRAIN_SIZE, 5)), end='')
    print("DT w/ Mean: ")
    print(printResult(DecisionTree.classifyMax(Preprocessing.normalization(dataM), TRAIN_SIZE, 5)), end='')

main()

Without max-depth: 
Without normalization:
DT w/ Deletion: 
Accurarcy:  0.6846986089644513
DT w/ Mean: 
Accurarcy:  0.7169230769230769
With normalization:
DT w/ Deletion: 
Accurarcy:  0.7418856259659969
DT w/ Mean: 
Accurarcy:  0.7261538461538461
With max-depth: 
Without normalization:
DT w/ Deletion: 
Accurarcy:  0.714064914992272
DT w/ Mean: 
Accurarcy:  0.703076923076923
With normalization:
DT w/ Deletion: 
Accurarcy:  0.714064914992272
DT w/ Mean: 
Accurarcy:  0.703076923076923


Wyniki są następujące, dla drzewa w wersji z 9 poziomami wyniki są lepsze niż dla tego z 5 poziomami. Przy 5 poziomach normalizacja nie miała znaczenia, gdzie w przypadku 9 poziomów
normalizacja pogorszyła wyniki. Ustalenie 9 poziomów wynikało z przetestowania metod z wieloma roznymi parametrami przy użyciu GridSearchCV. W tym wypadku usunięcie danych było lepszym wyjściem.

Kolejnym modelem jest Naiwny Bayes, dla którego kod wygląda następująco:

In [14]:
from sklearn.naive_bayes import GaussianNB
from Methods import NaiveBayes_KN as NBK

def classifyNB(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    classifier = GaussianNB()
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [15]:
def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfNB(dataD, dataM)

def getResultsOfNB(dataD, dataM):
    print("Without normalization:")
    print("NB w/ Deletion: ")
    print(printResult(NBK.classifyNB(dataD, TRAIN_SIZE)), end='')
    print("NB w/ Mean: ")
    print(printResult(NBK.classifyNB(dataM, TRAIN_SIZE)), end='')
    print("With normalization:")
    print("NB w/ Deletion: ")
    print(printResult(NBK.classifyNB(Preprocessing.normalization(dataD), TRAIN_SIZE)), end='')
    print("NB w/ Mean: ")
    print(printResult(NBK.classifyNB(Preprocessing.normalization(dataM), TRAIN_SIZE)), end='')

main()

Without normalization:
NB w/ Deletion: 
Accurarcy:  0.642967542503864
NB w/ Mean: 
Accurarcy:  0.6092307692307692
With normalization:
NB w/ Deletion: 
Accurarcy:  0.6290571870170015
NB w/ Mean: 
Accurarcy:  0.5984615384615385


Wyniki dla normalizacji jak i bez niej są bardzo podobne. Minimalnie na korzyść tych bez normalizacji. Lepsze wyniki wychodzą dla danych z uzupełnionymi brakami.

Kolejnym modelem jest kNN, dla którego kod wygląda następująco:

In [16]:
from sklearn.neighbors import KNeighborsClassifier

def classifyKN(df, train_size, neighbors):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    classifier = KNeighborsClassifier(n_neighbors=neighbors)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    accurarcy = classifier.score(test_set.iloc[:, :-1], test_set.iloc[:, -1])
    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

In [17]:

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))
    getResultsOfKNN(dataD, dataM)

def getResultsOfKNN(dataD, dataM):
    print("Without normalization:")
    for i in range(10, 13):
        print("KNN w/ Deletion: ")
        print(printResult(NBK.classifyKN(dataD, TRAIN_SIZE, i)), end='')
        print("KNN w/ Mean: ")
        print(printResult(NBK.classifyKN(dataM, TRAIN_SIZE, i)), end='')
    print("With normalization:")
    for i in range(10, 13):
        print("KNN w/ Deletion: ")
        print(printResult(NBK.classifyKN(Preprocessing.normalization(dataD), TRAIN_SIZE, i)), end='')
        print("KNN w/ Mean: ")
        print(printResult(NBK.classifyKN(Preprocessing.normalization(dataM), TRAIN_SIZE, i)), end='')

main()

Without normalization:
KNN w/ Deletion: 
Accurarcy:  0.6414219474497682
KNN w/ Mean: 
Accurarcy:  0.66
KNN w/ Deletion: 
Accurarcy:  0.6491499227202473
KNN w/ Mean: 
Accurarcy:  0.6507692307692308
KNN w/ Deletion: 
Accurarcy:  0.6537867078825348
KNN w/ Mean: 
Accurarcy:  0.6538461538461539
With normalization:
KNN w/ Deletion: 
Accurarcy:  0.7357032457496137
KNN w/ Mean: 
Accurarcy:  0.7461538461538462
KNN w/ Deletion: 
Accurarcy:  0.7372488408037094
KNN w/ Mean: 
Accurarcy:  0.7369230769230769
KNN w/ Deletion: 
Accurarcy:  0.7341576506955177
KNN w/ Mean: 
Accurarcy:  0.7215384615384616


Najlepsze wyniki są dla n=12, określiłem to przy użyciu tej pętli tutaj oraz również przy użyciu GridSearchCV. Przy użyciu normalizacji wyniki są zdecydowanie lepsze z nią. Wyniki są również
lepsze dla uzupełnienia braków średnią niż usunięciem danych.

Ostatnim modelem jest sieć neuronowa, dla której kod wygląda następująco:

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix



def classify(df, train_size):
    train_set, test_set = train_test_split(df, train_size=train_size, random_state=2137)

    # param_grid = {
    #     'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 50, 25), (10, 20, 30, 40), (100,50,100)],
    #     'activation': ['relu', 'tanh'],
    #     'solver': ['adam', 'sgd'],
    # }
    #
    # classifier = MLPClassifier()
    # grid_search = GridSearchCV(classifier, param_grid, cv=5)
    # grid_search.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])
    # print(grid_search.best_params_)
    # print(grid_search.best_score_)

    classifier = MLPClassifier(hidden_layer_sizes=(100, 50, 25), activation='tanh', solver='adam', max_iter=2000)
    classifier.fit(train_set.iloc[:, :-1], train_set.iloc[:, -1])

    predicted = classifier.predict(test_set.iloc[:, :-1])
    confusionMatrix = confusion_matrix(test_set.iloc[:, -1], predicted)
    accurarcy = accuracy_score(test_set.iloc[:, -1], predicted)
    return accurarcy, confusionMatrix

Użyty znowu został GridSearchCV, aby dobrać najlepsze parametry. Wyniki dla sieci neuronowej są bardzo dobre, a najlepsze są dla 100, 50, 25 neuronów w kolejnych warstwach.
Wyniki są następujące:

In [9]:
from Methods import Neural

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfNeural(dataD, dataM)


LAYERS = (10, 7)


def getResultsOfNeural(dataD, dataM):
    print("Without normalization:")
    print("Neural w/ Deletion: ")
    print(printResult(Neural.classify(dataD, TRAIN_SIZE, LAYERS)), end='')
    print("Neural w/ Mean: ")
    print(printResult(Neural.classify(dataM, TRAIN_SIZE, LAYERS)), end='')
    print("With normalization:")
    print("Neural w/ Deletion: ")
    print(printResult(Neural.classify(Preprocessing.normalization(dataD), TRAIN_SIZE, LAYERS)), end='')
    print("Neural w/ Mean: ")
    print(printResult(Neural.classify(Preprocessing.normalization(dataM), TRAIN_SIZE, LAYERS)), end='')

main()

Accurarcy:  0.7001545595054096
Neural w/ Mean: 
Accurarcy:  0.7461538461538462
With normalization:
Neural w/ Deletion: 
Accurarcy:  0.749613601236476
Neural w/ Mean: 
Accurarcy:  0.7830769230769231
Without normalization:
Neural w/ Deletion: 
Accurarcy:  0.7047913446676971
Neural w/ Mean: 
Accurarcy:  0.7261538461538461
With normalization:
Neural w/ Deletion: 


Wyniki są lepsze dla danych z normalizacją oraz dla uzupełnienia danych średnią.

Macierze błedów dla wszystkich modeli wyglądają następująco:

In [10]:

def main():
    dataD = Preprocessing.withDeletion(getData("./Data/winequality.csv"))
    dataM = Preprocessing.withMean(getData("./Data/winequality.csv"))

    getResultsOfDT(dataD, dataM)
    getResultsOfNB(dataD, dataM)
    getResultsOfKNN(dataD, dataM)
    getResultsOfNeural(dataD, dataM)

    return 0


def getData(directory):
    data = pd.read_csv(directory)
    data = shuffle(data)
    data['quality'] = data['quality'].replace([3, 4, 5], 'low')
    data['quality'] = data['quality'].replace([6, 7], 'medium')
    data['quality'] = data['quality'].replace([8, 9], 'high')
    return data


def printResult(result):
    print("Confusion Matrix: ")
    print(result[1])
    return ""

main()

Without max-depth: 
Without normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  5   1   6]
 [  2 159  78]
 [  8  75 313]]
DT w/ Mean: 
Confusion Matrix: 
[[  5   1   8]
 [  0 163  71]
 [  1  76 325]]
With normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  9   0  13]
 [  0 157  78]
 [  4  77 309]]
DT w/ Mean: 
Confusion Matrix: 
[[  3   0  15]
 [  1 166  75]
 [  5  85 300]]
With max-depth: 
Without normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  0   1  18]
 [  0 136 103]
 [  0  68 321]]
DT w/ Mean: 
Confusion Matrix: 
[[  1   0  23]
 [  0 157 102]
 [  1  66 300]]
With normalization:
DT w/ Deletion: 
Confusion Matrix: 
[[  0   1  18]
 [  0 136 103]
 [  0  68 321]]
DT w/ Mean: 
Confusion Matrix: 
[[  1   0  23]
 [  0 157 102]
 [  1  66 300]]
Without normalization:
NB w/ Deletion: 
Confusion Matrix: 
[[  4   2  13]
 [  1 138 100]
 [ 33  89 267]]
NB w/ Mean: 
Confusion Matrix: 
[[  9   2  13]
 [  1 145 113]
 [ 34  86 247]]
With normalization:
NB w/ Deletion: 
Confusion Matrix:

0

Podsumowująć w większości przypadków normalizacja danych daje lepsze wyniki, a uzupełnienie braków danych średnią daje lepsze wyniki niż samo usunięcie danych. Najlepszym modelem okazała się sieć neuronowa oraz kNN, gdzie wyniki były na poziomie 75%. Najgorszym modelem okazał się naiwny klasyfikator bayesowski, gdzie wyniki były na poziomie 60%. Wszystkie modele testowane były na 90% danych treningowych oraz 10% danych testowych, bo dla takich wyniki były najlepsze.